In [ ]:
%%bash
pip install --upgrade duckdb
pip install --upgrade jupysql
pip install --upgrade duckdb-engine

In [ ]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:////content/dmeyf.db

In [ ]:
%sql duckdb:///:default:

In [ ]:
%%sql
create or replace table competencia_01 as 
select 
    *
from read_csv_auto('./datasets/competencia_02.csv.gz')

In [ ]:
%%sql
CREATE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0);

In [ ]:
%%sql
CREATE MACRO calculo_ratio(a, b) AS NULLIF(a / NULLIF(b,0),0);

In [ ]:
%%sql
CREATE MACRO corrijo_drift_rank(a,b) AS rank() over (partition by b order by a);

#### Aplicando funciones para Competencia02

In [ ]:
%%sql
CREATE TABLE table_1 AS
SELECT
    *,
    
    suma_sin_null(ctarjeta_visa_transacciones, ctarjeta_master_transacciones) as tc_transacciones_total,
    suma_sin_null(ctarjeta_visa, ctarjeta_master) as tc_ctarjeta_total,
    suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo) as tc_consumo_total,
    suma_sin_null(Master_mconsumototal, Visa_mconsumototal) as tc_mconsumo_total,
    suma_sin_null(Visa_mlimitecompra, Master_mlimitecompra) as tc_limcompra_total,
    suma_sin_null(Master_mpagado, Visa_mpagado) as tc_pagado_total,
    suma_sin_null(Master_mpagominimo, Visa_mpagominimo) as tc_saldo_total,
    suma_sin_null(Master_msaldototal, Visa_msaldototal) as tc_pagomin_total,
    ifnull(mprestamos_personales, 0) + ifnull(mprestamos_prendarios, 0) + ifnull(mprestamos_hipotecarios, 0) as pasivos_total,
    ifnull(mplazo_fijo_dolares, 0) + ifnull(mplazo_fijo_pesos, 0) + ifnull(minversion1_pesos, 0) 
        + ifnull(minversion1_dolares, 0) + ifnull(minversion2, 0) as activos_total,
    ifnull(mpayroll, 0) + ifnull(mpayroll2, 0) as flujos_ingresos_total,
    ifnull(mcajeros_propios_descuentos, 0) + ifnull(mtarjeta_visa_descuentos, 0) + ifnull(mtarjeta_master_descuentos, 0) as beneficios_total,
    ifnull(mcomisiones_mantenimiento, 0) + ifnull(mcomisiones_otras, 0) as comisiones_total,
    ifnull(mcuenta_debitos_automaticos, 0) + ifnull(mttarjeta_visa_debitos_automaticos, 0) + ifnull(mttarjeta_master_debitos_automaticos, 0) 
        + ifnull(mpagodeservicios, 0) + ifnull(mpagomiscuentas, 0) as gastos_total,


    corrijo_drift_rank(tc_mconsumo_total, foto_mes) as tc_mconsumo_total_sin_drift,
    corrijo_drift_rank(tc_limcompra_total, foto_mes) as tc_limcompra_total_sin_drift,
    corrijo_drift_rank(tc_pagado_total, foto_mes) as tc_pagado_total_sin_drift,
    corrijo_drift_rank(pasivos_total, foto_mes) as pasivos_total_sin_drift,
    corrijo_drift_rank(activos_total, foto_mes) as activos_total_sin_drift,
    corrijo_drift_rank(flujos_ingresos_total, foto_mes) as flujos_ingresos_total_sin_drift,
    corrijo_drift_rank(comisiones_total, foto_mes) as comisiones_total_sin_drift,
    corrijo_drift_rank(gastos_total, foto_mes) as gastos_total_sin_drift,
    


    calculo_ratio(tc_consumo_total, flujos_ingresos_total) as consumo_sobre_ingresos,
    calculo_ratio(tc_mconsumo_total, flujos_ingresos_total) as tcm_consumo_sobre_ingresos,
    calculo_ratio(tc_pagado_total, flujos_ingresos_total) as pagado_sobre_ingresos,
    calculo_ratio(tc_pagado_total, activos_total) as pagado_sobre_activos,
    calculo_ratio(tc_pagomin_total, flujos_ingresos_total) as pagomin_sobre_ingresos,
    calculo_ratio(tc_consumo_total, activos_total) as gastos_sobre_activos,
    calculo_ratio(activos_total, pasivos_total) as activos_sobre_pasivos,
    calculo_ratio(beneficios_total, comisiones_total) as beneficios_sobre_comisiones,
    calculo_ratio(beneficios_total, tc_consumo_total) as beneficios_sobre_consumos,
    calculo_ratio(gastos_total, flujos_ingresos_total) as gastos_sobre_ingresos,
    calculo_ratio(tc_limcompra_total, tc_consumo_total) as lim_sobre_consumos,
    
     
FROM competencia_01

In [ ]:
%%sql
DROP TABLE IF EXISTS competencia_01

In [ ]:
%%sql
CREATE TABLE table_2 AS
SELECT
    *,

    lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as ctrx_quarter_sin_drift,
    lag(tc_mconsumo_total_sin_drift, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_tc_mconsumo_total_sin_drift,
    lag(tc_limcompra_total_sin_drift, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_tc_limcompra_total_sin_drift,
    lag(tc_pagado_total_sin_drift, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_tc_pagado_total_sin_drift,
    lag(pasivos_total_sin_drift, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_pasivos_total_sin_drift,
    lag(activos_total_sin_drift, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_activos_total_sin_drift,
    lag(flujos_ingresos_total_sin_drift, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_flujos_ingresos_total_sin_drift,
    lag(comisiones_total_sin_drift, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_comisiones_total_sin_drift,
    lag(gastos_total_sin_drift, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_gastos_total_sin_drift,
    
     
FROM table_1

In [ ]:
%%sql
DROP TABLE IF EXISTS table_1

In [ ]:
%%sql
CREATE TABLE table_3 AS
SELECT
    *,
        
    avg(ctrx_quarter) over ventana_6 as ctrx_quarter_media_6, 
    max(ctrx_quarter) over ventana_6 as ctrx_quarter_max_6, 
    min(ctrx_quarter) over ventana_6 as ctrx_quarter_min_6,
    regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_6 as ctrx_quarter_slope_6,
    
    avg(cproductos) over ventana_6 as cproductos_media_6, 
    max(cproductos) over ventana_6 as cproductos_max_6, 
    min(cproductos) over ventana_6 as cproductos_min_6,
    regr_slope(cproductos, cliente_antiguedad) over ventana_6 as cproductos_slope_6,
    
    avg(ctarjeta_debito_transacciones) over ventana_6 as ctarjeta_debito_transacciones_media_6, 
    max(ctarjeta_debito_transacciones) over ventana_6 as ctarjeta_debito_transacciones_max_6, 
    min(ctarjeta_debito_transacciones) over ventana_6 as ctarjeta_debito_transacciones_min_6,
    regr_slope(ctarjeta_debito_transacciones, cliente_antiguedad) over ventana_6 as ctarjeta_debito_transacciones_slope_6,

    avg(chomebanking_transacciones) over ventana_6 as chomebanking_transacciones_media_6, 
    max(chomebanking_transacciones) over ventana_6 as chomebanking_transacciones_max_6, 
    min(chomebanking_transacciones) over ventana_6 as chomebanking_transacciones_min_6,
    regr_slope(chomebanking_transacciones, cliente_antiguedad) over ventana_6 as chomebanking_transacciones_slope_6,

    avg(tc_transacciones_total) over ventana_6 as tc_transacciones_total_media_6, 
    max(tc_transacciones_total) over ventana_6 as tc_transacciones_total_max_6, 
    min(tc_transacciones_total) over ventana_6 as tc_transacciones_total_min_6,
    regr_slope(tc_transacciones_total, cliente_antiguedad) over ventana_6 as tc_transacciones_total_slope_6,    
     
FROM table_2
window ventana_6 as (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row)

In [ ]:
%%sql
DROP TABLE IF EXISTS table_2

In [1]:
%%sql
ALTER TABLE table_3 DROP COLUMN ctarjeta_visa_transacciones;
ALTER TABLE table_3 DROP COLUMN ctarjeta_master_transacciones;
ALTER TABLE table_3 DROP COLUMN ctarjeta_visa;
ALTER TABLE table_3 DROP COLUMN ctarjeta_master;
ALTER TABLE table_3 DROP COLUMN mtarjeta_visa_consumo;
ALTER TABLE table_3 DROP COLUMN mtarjeta_master_consumo;
ALTER TABLE table_3 DROP COLUMN Master_mconsumototal;
ALTER TABLE table_3 DROP COLUMN Visa_mconsumototal;
ALTER TABLE table_3 DROP COLUMN Visa_mlimitecompra;
ALTER TABLE table_3 DROP COLUMN Master_mlimitecompra;
ALTER TABLE table_3 DROP COLUMN Master_mpagado;
ALTER TABLE table_3 DROP COLUMN Visa_mpagado;
ALTER TABLE table_3 DROP COLUMN Master_mpagominimo;
ALTER TABLE table_3 DROP COLUMN Visa_mpagominimo;
ALTER TABLE table_3 DROP COLUMN tc_mconsumo_total;
ALTER TABLE table_3 DROP COLUMN tc_limcompra_total;
ALTER TABLE table_3 DROP COLUMN tc_pagado_total;
ALTER TABLE table_3 DROP COLUMN pasivos_total;
ALTER TABLE table_3 DROP COLUMN activos_total;
ALTER TABLE table_3 DROP COLUMN flujos_ingresos_total;
ALTER TABLE table_3 DROP COLUMN comisiones_total;
ALTER TABLE table_3 DROP COLUMN gastos_total;


UsageError: Cell magic `%%sql` not found.


In [ ]:
%%sql
copy table_3 to './datasets/competencia_02.csv.gz' (FORMAT CSV, HEADER)